In [ ]:
import torch # Import the main PyTorch library
import torch.nn as nn # Import the neural network module from PyTorch
import torch.optim as optim # Import optimization algorithms from PyTorch
import torchvision # Import torchvision for datasets and models
import torchvision.transforms as transforms # Import image transformations
import torch.nn.functional as F # Import functional interface for neural network operations

In [ ]:
# Define a sequence of image transformations to be applied to the dataset
transform = transforms.Compose(
    [transforms.ToTensor(), # Convert images from PIL Image or NumPy ndarray to PyTorch tensors
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # Normalize the tensor image with mean and standard deviation

# Load the CIFAR10 training dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, # Specify the root directory for data, indicate it's the training set
                                        download=True, transform=transform) # Download data if not present, apply the defined transformations

# Create a DataLoader for the training dataset
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, # Load data in batches of 4 samples
                                          shuffle=True, num_workers=2) # Shuffle data at each epoch, use 2 subprocesses for data loading

# Load the CIFAR10 test dataset
testset = torchvision.datasets.CIFAR10(root='./data', train=False, # Specify the root directory for data, indicate it's the test set
                                       download=True, transform=transform) # Download data if not present, apply the defined transformations

# Create a DataLoader for the test dataset
testloader = torch.utils.data.DataLoader(testset, batch_size=4, # Load data in batches of 4 samples
                                         shuffle=False, num_workers=2) # Do not shuffle test data, use 2 subprocesses for data loading

# Define the class names for the CIFAR10 dataset
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170M/170M [00:03<00:00, 47.5MB/s]


In [ ]:
# Define the Convolutional Neural Network (CNN) architecture
class Net(nn.Module): # Inherit from nn.Module, the base class for all neural network modules
    def __init__(self): # Constructor for the network
        super(Net, self).__init__() # Call the constructor of the parent class (nn.Module)
        self.conv1 = nn.Conv2d(3, 6, 5) # First convolutional layer: 3 input channels (for RGB images), 6 output channels, 5x5 kernel size
        self.pool = nn.MaxPool2d(2, 2) # Max pooling layer: 2x2 window, stride of 2
        self.conv2 = nn.Conv2d(6, 16, 5) # Second convolutional layer: 6 input channels, 16 output channels, 5x5 kernel size
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # First fully connected (linear) layer: input features from flattened conv2 output, 120 output features
        self.fc2 = nn.Linear(120, 84) # Second fully connected layer: 120 input features, 84 output features
        self.fc3 = nn.Linear(84, 10) # Third fully connected layer: 84 input features, 10 output features (for 10 classes)

    def forward(self, x): # Defines the forward pass of the network
        x = self.pool(F.relu(self.conv1(x))) # Apply conv1, then ReLU activation, then max pooling
        x = self.pool(F.relu(self.conv2(x))) # Apply conv2, then ReLU activation, then max pooling
        x = x.view(-1, 16 * 5 * 5) # Reshape the tensor for the fully connected layers (flatten operation)
        x = F.relu(self.fc1(x)) # Apply fc1, then ReLU activation
        x = F.relu(self.fc2(x)) # Apply fc2, then ReLU activation
        x = self.fc3(x) # Apply fc3 (output layer)
        return x # Return the output predictions


net = Net() # Create an instance of the Net class

In [ ]:
criterion = nn.CrossEntropyLoss() # Define the loss function: Cross-Entropy Loss, suitable for multi-class classification
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # Define the optimizer: Stochastic Gradient Descent (SGD) with learning rate 0.001 and momentum 0.9

In [ ]:
for epoch in range(2): # Loop over the dataset multiple times for a specified number of epochs (here, 2 epochs)

    running_loss = 0.0 # Initialize running loss for tracking average loss per mini-batch
    for i, data in enumerate(trainloader, 0): # Iterate over batches of data from the training data loader
        inputs, labels = data # Get the input images and their corresponding labels from the batch

        optimizer.zero_grad() # Zero the gradients of all optimized tensors to prevent accumulation from previous iterations

        outputs = net(inputs) # Perform a forward pass: get predictions from the network for the current inputs
        loss = criterion(outputs, labels) # Calculate the loss between the network's outputs and the true labels
        loss.backward() # Perform a backward pass: compute gradients of the loss with respect to all learnable parameters
        optimizer.step() # Update the network's parameters using the calculated gradients

        running_loss += loss.item() # Accumulate the loss for the current batch
        if i % 2000 == 1999: # Check if 2000 mini-batches have been processed
            print('[%d, %5d] loss: %.3f' % # Print the current epoch, batch number, and average loss
                  (epoch + 1, i + 1, running_loss / 2000)) # Format and display the information
            running_loss = 0.0 # Reset running loss after printing

print('Finished Training') # Indicate that the training process has completed

[1,  2000] loss: 2.287
[1,  4000] loss: 1.920
[1,  6000] loss: 1.693
[1,  8000] loss: 1.583
[1, 10000] loss: 1.515
[1, 12000] loss: 1.464
[2,  2000] loss: 1.380
[2,  4000] loss: 1.355
[2,  6000] loss: 1.355
[2,  8000] loss: 1.310
[2, 10000] loss: 1.291
[2, 12000] loss: 1.284
Finished Training


In [ ]:
correct = 0 # Initialize counter for correctly predicted samples
total = 0 # Initialize counter for total samples processed
with torch.no_grad(): # Disable gradient calculation for inference, saves memory and computation
    for data in testloader: # Iterate over batches of data from the test data loader
        images, labels = data # Get the input images and their corresponding labels from the batch
        outputs = net(images) # Perform a forward pass: get predictions from the network for the test images
        _, predicted = torch.max(outputs.data, 1) # Get the index of the class with the highest probability (the prediction)
        total += labels.size(0) # Add the number of samples in the current batch to the total count
        correct += (predicted == labels).sum().item() # Count how many predictions were correct in the current batch

print('Accuracy of the network on the 10000 test images: %d %%' % ( # Print the overall accuracy
    100 * correct / total)) # Calculate and format the accuracy percentage

Accuracy of the network on the 10000 test images: 55 %
